Feature Engineering first: generating more features based on all the features we have now

In [1]:
from sklearn import preprocessing
import pandas as pd

In [55]:
'''
data: the raw input data we have 
'''
def Generate_Features(data):
    
    # encoding the department description label by sklearn
    le = preprocessing.LabelEncoder()
    # encode and transform the department description label
    data['DepartmentDescription'] = le.fit_transform(list(data['DepartmentDescription']))
    data['Weekday'] = preprocessing.LabelEncoder().fit_transform(list(data['Weekday']))
     # assign a new column with scancount as the base value
    data['Count'] = data['ScanCount']
    data['Count'][data['ScanCount']<0] = 0  # filter the negative values
    data['FinelineNumber'].fillna(value = 10000, inplace = True) # replace the na values with 10000
    data['Upc'].fillna(value = -9999, inplace = True)# replace the na values with -9999
    #===================== Missing Value Indicators ======================================#
    # null value exist in Department Description, encoded as 67
    data1 = data[data['DepartmentDescription'] == 67]
    # the number of non na observations of each visit number
    data1 = data[data['DepartmentDescription']==67]
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'Count_Null'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True) 
    data['Count_Null'].fillna(value=0, inplace=True) 
    data['Count_Null'][data['Count_Null']>0] = 1 # 把count 换成1

    data1 = data[data['ScanCount']<0]
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'ScanCount_Neg'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True) 
    data['ScanCount_Neg'].fillna(value=0, inplace=True)
    data['ScanCount_Neg'][data['ScanCount_Neg']>0] = 1
    
    data1 = data[data['FinelineNumber']==10000]
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'FinelineNumber_Missing'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True) 
    data['FinelineNumber_Missing'].fillna(value=0, inplace=True)
    data['FinelineNumber_Missing'][data['FinelineNumber_Missing']>0] = 1

    data1 = data.groupby(['VisitNumber',  'FinelineNumber'], as_index=False)['Count'].count()
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'N_Fineline'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True)
         
    data1 = data.groupby(['VisitNumber',  'Upc'], as_index=False)['Count'].count()
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'N_Upc'}, inplace=True) 
    data = data.merge( data1, how='left', on=['VisitNumber'], copy=True) 

    data1 = data.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].count()
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'N_Dep'}, inplace=True) 
    data = data.merge( data1, how='left', on=['VisitNumber'], copy=True)
    
    # group data for new features:
    # 1. visit number and departmant description
    # the scan counts for each visitnumber and department combination
    temp1 = data.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['ScanCount'].sum()
    temp11 = temp1.groupby(['VisitNumber'], as_index=False)['ScanCount'].min()
    temp12 = temp1.groupby(['VisitNumber'], as_index=False)['ScanCount'].max()
    temp13 = temp1.groupby(['VisitNumber'], as_index=False)['ScanCount'].mean()
    temp11.rename(columns={'ScanCount': 'Min_Count'}, inplace=True)
    temp12.rename(columns={'ScanCount': 'Max_Count'}, inplace=True)
    temp13.rename(columns={'ScanCount': 'Mean_Count'}, inplace=True)
    # left join to the dataset
    data = data.merge(temp11, how='left', on=['VisitNumber'], copy=True)
    data = data.merge(temp12, how='left', on=['VisitNumber'], copy=True)
    data = data.merge(temp13, how='left', on=['VisitNumber'], copy=True)  
    
    # 2. UPC: A UPC should have 12 digits. The first 6 digits are company code. The next four are item code.
    # add check sum to the end of every upc and missing zeros at the begining of the upc
    # convert Upc to string first
    data['Upc'] = data['Upc']*10
    data['Upc'] = data.Upc.apply(string_convert)
    data['Upc_full'] = data.Upc.apply(upc_fullfill)
    data['company'] =  data.Upc_full.apply(company_extractor)
    return data

def string_convert(x):
    return ('%.2f' % (x,)).rstrip('0').rstrip('.')

def upc_checksum_calculator(x):
    try:
        odd = map(int, ','.join(x[-1::-2]).split(','))
        even = map(int, ','.join(x[-2::-2]).split(','))
        sum_odd = sum(odd) * 3
        total = sum_odd + sum(even)
        rest = total % 10
        if rest == 0:
            return rest
        return 10 - rest
    except:
        return -9999 # return na for upc which can not be decoded
    
def upc_fullfill(x):
    try:
        if len(x) < 12:
            missing_zeros = 11 - len(x)
            zeros = ['0'] * missing_zeros
            full_upc = zeros + ','.join(x).split(',') + [str(upc_checksum_calculator(x))]
            full_upc = ''.join(full_upc)
            return full_upc
        else:
            return x
    except:
        return -9999
    
def company_extractor(x):
    try:
        p = x[:6]
        if p == '000000':
            return x[-5]
        return p
    except:
        return -9999

In [56]:
path = 'C:\Users\shuyi\Documents\StudyResource\Kaggle\\'
train_file = "train.csv"
train_data = pd.read_csv(path + train_file)

In [ ]:
train_data_step2 = Generate_Features(train_data)

In [58]:
train_data_step2[10:20]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Count_Null,ScanCount_Neg,FinelineNumber_Missing,N_Fineline,N_Upc,N_Dep,Min_Count,Max_Count,Mean_Count,Upc_full,company
10,26,8,0,52000102390,1,17,4606.0,1,0.0,1.0,1.0,17,21,7,1,18,4.0,520001023902,520001
11,26,8,0,886793005010,2,49,3504.0,2,0.0,1.0,1.0,17,21,7,1,18,4.0,886793005010,886793
12,26,8,0,220060000000,1,41,6009.0,1,0.0,1.0,1.0,17,21,7,1,18,4.0,220060000000,220060
13,26,8,0,22367604520,1,49,7.0,1,0.0,1.0,1.0,17,21,7,1,18,4.0,223676045209,223676
14,26,8,0,886793005010,-1,49,3504.0,0,0.0,1.0,1.0,17,21,7,1,18,4.0,886793005010,886793
15,26,8,0,22384002000,2,49,3565.0,2,0.0,1.0,1.0,17,21,7,1,18,4.0,223840020001,223840
16,26,8,0,30192942030,1,49,2801.0,1,0.0,1.0,1.0,17,21,7,1,18,4.0,301929420307,301929
17,26,8,0,724504088400,1,49,1028.0,1,0.0,1.0,1.0,17,21,7,1,18,4.0,724504088400,724504
18,26,8,0,255415000000,2,16,1305.0,2,0.0,1.0,1.0,17,21,7,1,18,4.0,255415000000,255415
19,26,8,0,23100107760,1,51,3300.0,1,0.0,1.0,1.0,17,21,7,1,18,4.0,231001077603,231001


In [59]:
# save the data 
train_data_step2.to_csv(path + "step2.csv")